In [26]:
import pandas as pd
import numpy as np

Importamos los archivos como tablas

In [27]:
amazon = pd.read_csv('datasets/amazon_prime_titles.csv',dtype = {'show_id': str},delimiter = ',',encoding = "utf-8")
disney = pd.read_csv('datasets/disney_plus_titles.csv',dtype = {'show_id': str},delimiter = ',',encoding = "utf-8")
hulu = pd.read_csv('datasets/hulu_titles.csv',dtype = {'show_id': str},delimiter = ',',encoding = "utf-8")
netflix = pd.read_json('datasets/netflix_titles.json',encoding = "utf-8")

Se le añade la columna de plataforma

In [28]:
amazon["Plataforma"] = "Amazon"
netflix["Plataforma"] = "Netflix"
hulu["Plataforma"] = "Hulu"
disney["Plataforma"] = "Disney"

Concatenamos las tablas

In [29]:
frames = [amazon,disney,hulu,netflix]
movies = pd.concat(frames, ignore_index=True)

Eliminamos las columnas que no necesitamos

In [30]:
movies = movies.drop("country", axis = 1)
movies = movies.drop("description", axis = 1)
movies = movies.drop("date_added", axis = 1)
movies = movies.drop("director", axis = 1)
movies = movies.drop("rating", axis = 1)
movies = movies.drop("show_id", axis = 1)


In [32]:
movies.head()

,type,title,cast,release_year,duration,listed_in,Plataforma
0,Movie,The Grand Seduction,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",2014,113 min,"Comedy, Drama",Amazon
1,Movie,Take Care Good Night,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",2018,110 min,"Drama, International",Amazon
2,Movie,Secrets of Deception,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",2017,74 min,"Action, Drama, Suspense",Amazon
3,Movie,Pink: Staying True,"Interviews with: Pink, Adele, Beyoncé, Britney...",2014,69 min,Documentary,Amazon
4,Movie,Monster Maker,"Harry Dean Stanton, Kieran O'Brien, George Cos...",1989,45 min,"Drama, Fantasy",Amazon


Normalizamos los datos faltantes


In [33]:
movies["cast"] = movies["cast"].replace(np.nan, "Sin Datos")
movies["duration"] = movies["duration"].replace(np.nan, 0)
movies["title"] = movies["title"].replace(np.nan, "Sin Datos")
movies = movies.fillna("Sin Datos")


Extraemos los valores con la funcion

In [34]:
movies['duration'] = movies['duration'] .replace({'[a-zA-Z ]':''}, regex=True)

Normalizamos el tipo de dato


In [35]:
movies["duration"] = pd.to_numeric(movies["duration"])
movies["type"] = movies["type"].astype("string")
movies["title"] = movies["title"].astype("string")
movies["cast"] = movies["cast"].astype("string")
movies["Plataforma"] = movies["Plataforma"].astype("string")




Creamos una nueva columna con el tipo de dato de duracion ya sea min o season

In [36]:
movies["duration_type"] =  np.where(movies["type"] == "Movie", "min", "season")

Cambiamos nombres de columnas

In [37]:
movies.rename(columns= {"type": "Tipo", "title" : "Titulo", "cast": "Actores", "release_year" : "Anio", "duration" : "Duracion", "listed_in": "Genero", "duration_type" : "Duracion_tipo"} , inplace= True)

In [11]:
movies.head()

,Tipo,Titulo,Actores,Anio,Duracion,Genero,Plataforma,Titulo,Duracion_tipo
0,Movie,The Grand Seduction,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",2014,113,"Comedy, Drama",Amazon,113 min,min
1,Movie,Take Care Good Night,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",2018,110,"Drama, International",Amazon,110 min,min
2,Movie,Secrets of Deception,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",2017,74,"Action, Drama, Suspense",Amazon,74 min,min
3,Movie,Pink: Staying True,"Interviews with: Pink, Adele, Beyoncé, Britney...",2014,69,Documentary,Amazon,69 min,min
4,Movie,Monster Maker,"Harry Dean Stanton, Kieran O'Brien, George Cos...",1989,45,"Drama, Fantasy",Amazon,45 min,min


In [80]:
x = movies.loc[movies["Anio"] == 2010][movies["Tipo"] == "TV Show"][movies["Plataforma"] == "Hulu"]["Duracion"].max()

C:\Users\LEGION\AppData\Local\Temp\ipykernel_10520\2211526773.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  x = movies.loc[movies["Anio"] == 2010][movies["Tipo"] == "TV Show"][movies["Plataforma"] == "Hulu"]["Duracion"].max()


In [53]:
movies["Duracion"] = movies["Duracion"].replace(np.nan, 0)

In [82]:
check_nan = movies["Plataforma"].isnull().values.any()
print(check_nan)

False


In [93]:
x =movies.loc[(movies["Anio"] == int(1930)) & (movies["Tipo"] == "Movie") & (movies["Plataforma"] == "Netflix")]["Duracion"].max()

In [ ]:

def get_max_duration(anio,plataforma,tipo):
    #Convierto anio en int en caso de recibirlo como una string
    anio = int(anio)
    #Creo mis tuplas con mis posibles opciones para verificar que el input se encuentre dentro de las mismas.
    tuple_plataformas = ("amazon","disney","hulu","netflix")
    tuple_tipo = ("min","season")
    #Doy las condiciones que deben ser cumplidas para entrar al análisis, de no ser así, pido que se verifiquen los datos ingresados.
    try:
        if (anio in range(movies.min(),movies.max()+1)) and (plataforma in (tuple_plataformas)) and (tipo in tuple_tipo):
            #Busco el ID de la película/serie con mayor duración que se encuentre en el Dataframe que me devuelve la Query cuyo año, plataforma y tipo recibo como parámetros.
            id_final = movies.query(f"release_year == {anio} and platform == '{plataforma}' and duration_type == '{tipo}'").Duracion.idxmax()
            #Utilizo la ID recibida para crear un diccionario con los datos del dataframe en esa ID, luego extraigo los values que me interesan informarle al usuario que realiza la consulta.
            return f'{movies[id_final:id_final+1].to_dict().get("title").get(id_final)} Tiene la mayor duración: {movies[id_final:id_final+1].to_dict().get("duration").get(id_final)}'
        else:
            return f"Faltan datos para su consulta o alguno de los parámetros es incorrecto, por favor verifique que los datos ingresados."
    except ValueError:
            return f"La plataforma {plataforma} no posee registros en {anio}"

In [95]:
y = (movies.loc[(movies["Anio"] == 2019) & (movies["Tipo"] == "Movie") & (movies["Plataforma"] == "Netflix") & (movies["Duracion"] == 150)]["Titulo"]).item()

ValueError: can only convert an array of size 1 to a Python scalar

Armamos la primera funcion: Máxima duración según tipo de film (película/serie), por plataforma y por año: El request debe ser: get_max_duration(año, plataforma, [min o season])

In [73]:
def get_max_duration(anio, plataforma, min_o_season):
    #if min_o_season == "min":
        #x = movies[movies["Anio"] == anio][movies["Tipo"] == "Movie"][movies["Plataforma"] == plataforma]["Duracion"].max()
    if min_o_season == "min":
        x = movies.loc[(movies["Anio"] == int(anio)) & (movies["Tipo"] == "Movie") & (movies["Plataforma"] == plataforma)]["Duracion"].max()
        

        #y = (movies.loc[(movies["Anio"] == int(anio)) & (movies["Tipo"] == "Movie") & (movies["Plataforma"] == plataforma) & (movies["Duracion"] == int(x))]["Titulo"])
        #return(print(y, "tiene la mayor duracion :", int(x), min_o_season))
        #return print("la duracion maxima en minutos para el ano", anio, "y la plataforma", plataforma , "fue de", x, min_o_season,"/s")

    if min_o_season == "season":
        x = movies.loc[(movies["Anio"] == int(anio)) & (movies["Tipo"] == "TV Show") & (movies["Plataforma"] == plataforma)]["Duracion"].max()
        movies(np.isnan(x) = 0
        print( x)
        #y = (movies.loc[(movies["Anio"] == int(anio)) & (movies["Tipo"] == "TV Show") & (movies["Plataforma"] == plataforma) & (movies["Duracion"] == int(x))]["Titulo"]).item()
        #return(print(y, "tiene la mayor duracion :", x, min_o_season))
    
        
    

In [74]:
get_max_duration(1930, "Netflix", "season")

TypeError: 'float' object does not support item assignment

Armamos la segunda funcion: Cantidad de películas y series (separado) por plataforma El request debe ser: get_count_plataform(plataforma)

In [ ]:
def get_count_platform(plataforma):

    x =  movies[(movies["Tipo"] == "Movie") & (movies["Plataforma"] == plataforma )]["Titulo"].count()
    y =  movies[(movies["Tipo"] == "TV Show") & (movies["Plataforma"] == plataforma )]["Titulo"].count()

    return( print("La plataforma", plataforma, "tiene",x,"peliculas","y", y, "series"))

In [ ]:
get_count_platform("Hulu")

La plataforma Hulu tiene 1484 peliculas y 1589 series


Cantidad de veces que se repite un género y plataforma con mayor frecuencia del mismo. El request debe ser: get_listedin('genero')

In [ ]:
def get_listedin(genero):

    a = 0

    for i in movies[(movies["Plataforma"] == "Amazon")]["Genero"]: 
            
        x = list(i.split(","))

        lista= []
        for names in x:
            lista.append(names.replace(" ",""))
        
        if genero in lista:
            a+=1

    
    b = 0

    for i in movies[(movies["Plataforma"] == "Hulu")]["Genero"]: 
            
        
        x = list(i.split(","))
        
        
        lista= []
        for names in x:
            lista.append(names.replace(" ",""))
        


        if genero in lista:
            b+=1

    
    c = 0

    for i in movies[(movies["Plataforma"] == "Netflix")]["Genero"]: 
            
        
        x = list(i.split(","))
        
        
        lista= []
        for names in x:
            lista.append(names.replace(" ",""))
        


        if genero in lista:
            c+=1
    d = 0

    for i in movies[(movies["Plataforma"] == "Disney")]["Genero"]: 
            
        
        x = list(i.split(","))
        
        
        lista= []
        for names in x:
            lista.append(names.replace(" ",""))
        


        if genero in lista:
            d+=1
    return print("El Genero", genero,"aparece",a,"veces en Amazon,",b ,"veces en Hulu,", c, "veces en Netflix y", d, "veces en Disney")
    


In [ ]:
get_listedin("Documentary")

El Genero Documentary aparece 993 veces en Amazon, 0 veces en Hulu, 0 veces en Netflix y 174 veces en Disney


Actor que más se repite según plataforma y año. El request debe ser: get_actor(plataforma, año)

In [ ]:

def get_actor(plataforma, anio):

    lista = []


    for i in movies[(movies["Plataforma"] == plataforma) & (movies["Anio"] == anio)]["Actores"]:
        x = list(i.split(","))
        
        for n in x:
            lista.append(n.strip())



    actoresunicos = []
    numero=[]
    mayor = 0
    for n in lista:
        if n == "Sin Datos":
            continue
        if n  in actoresunicos:
            indice = actoresunicos.index(n)
            numero[indice] += 1
        
        if not n in actoresunicos:
            actoresunicos.append(n)
            numero.append(1)
        #if actor in actoresunicos:
            #indice = lista.index(actor)
            #index[indice] +=1


            #actoresunicos.append(n)

    for valor in numero:
        if valor > mayor:
            mayor = valor
    actorunico = actoresunicos[numero.index(mayor)]               
    print("La actriz con mas apariciones para el año", anio, "en la plataforma", plataforma, "es", actorunico, "con", mayor, "apariciones" )
            
                
 

            
#print(actoresunicos)              
#print(numero)

    #for names in x:
        #lista.append(names.replace(" ",""))
   

In [ ]:
get_actor("Amazon", 2017
)

La actriz con mas apariciones para el año 2017 en la plataforma Amazon es Maggie Binkley con 16 apariciones
